In [1]:
import datetime
import math
import time
import itertools
from contextlib import closing
from collections import OrderedDict
import json
import numpy as np
import pandas as pd
import MySQLdb
import pymssql
from sqlalchemy import create_engine
import dfutils
import matplotlib.pyplot as plt

In [2]:
#DB_NAME = 'wic'
#WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
#DB_USER = 'aduprey'
#DB_PASSWORD = 'aduprey'
WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
DB_USER = 'root'
DB_PASSWORD = 'waterislandcapital'
DB_NAME = 'wic'
engine = create_engine("mysql://" + DB_USER + ":" + DB_PASSWORD + "@" + WIC_DB_HOST + "/wic")

In [3]:
#pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 50)

In [4]:
def optimized_execute(query, commit=False):
    '''
    Executes the search query in Northpoint
    '''
    
    with closing(pymssql.connect(host='10.16.1.16', user='readonly_user', password='readonly_user',
                                 database='PnLAppDb')) as np_pnl_conn:
        with closing(np_pnl_conn.cursor()) as np_pnl_cursor:
            np_pnl_cursor.execute(query)
            if commit:
                np_pnl_conn.commit()
                return

            fetched_res = np_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
            return fetched_res

In [5]:
def tradar_optimized_execute(query, commit=False):
    try:
        with closing(pymssql.connect(host='NYDC-WTRTRD01', user='paz', password='Welcome2',
                                     database='TradarBE')) as trdr_pnl_conn:
            with closing(trdr_pnl_conn.cursor()) as trdr_pnl_cursor:
                trdr_pnl_cursor.execute(query)
                if commit:
                    trdr_pnl_conn.commit()
                    return

                fetched_res = trdr_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
                return fetched_res
    except Exception as e:
        print e

In [6]:
def wic_optimized_execute(query, commit=False, retrieve_column_names=False, connection_timeout=250, extra_values=None):
    with closing(MySQLdb.connect(host=WIC_DB_HOST, user=DB_USER, passwd=DB_PASSWORD, db=DB_NAME)) as wic_cnx:
        with closing(wic_cnx.cursor()) as wic_cursor:

            if extra_values is not None:
                wic_cursor.execute(query, (extra_values,))
            else:
                wic_cursor.execute(query)
            if commit:
                wic_cnx.commit()
                return
            fetched_res = wic_cursor.fetchall()  # fetch (and discard) remaining rows

            if retrieve_column_names:
                return fetched_res, [i[0] for i in wic_cursor.description]

            return fetched_res

In [7]:
def get_position_calculated_values_history_max_date():  # done
    query = "SELECT MAX(PCVH.TradeDate) FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH "
    results = optimized_execute(query)
    if len(results) == 0:
        return None

    return results[0][0]

In [8]:
def get_position_calculated_values_history(start_date_yyyy_mm_dd=None, end_date_yyyy_mm_dd=None,
                                           limit_to_tradegroups=[], is_tg_names_in_tradar_format=False,
                                           fund_code=None, NP_qty_rollup=True):
    if start_date_yyyy_mm_dd is None or end_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '2016-01-01'
        end_date_yyyy_mm_dd = get_position_calculated_values_history_max_date()
    tg_filter_clause = ""
    if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:
        tg_filter_clause = " AND PCVH.TradeGroup IN (" + ",".join([("'" + tg + "'") 
                                                                   for tg in limit_to_tradegroups 
                                                                   if tg is not None]) + ") "
        if is_tg_names_in_tradar_format: 
            tg_filter_clause = tg_filter_clause.replace('PCVH.TradeGroup', 'SUBSTRING(PCVH.TradeGroup,0,21)')
            
    fund_code_filtuer_clause = " " if fund_code is None else " AND PCVH.FundCode = '" + fund_code + "' "
    # region query
    # New Fund by added bv Kshitij "WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL":"MACO"
    query = "SELECT " \
            "PCVH.TradeDate, " \
            "ISNULL(PCVH.FundCode, " \
            "CASE F.FundName " \
            "WHEN 'Columbia' THEN 'CAM' " \
            "WHEN 'Litman Gregory' THEN 'LG' " \
            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
            "WHEN 'TransAmerica' THEN 'TAF' " \
            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'WED' " \
            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL' THEN 'MACO' " \
            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
            "END " \
            ") AS FundCode, " \
            "SPT_TRDGRP.SecName AS TradeGroup, " \
            "SPT_TRDGRP.SecName AS TradeGroup_Tradar_Name, " \
            "PCVH.[Marketing GROUP] AS Sleeve,   " \
            "ISNULL(PCVH.[Bucket],'NA') AS Bucket, " \
            "PCVH.TradeGroupId, " \
            "SPT.Ticker, " \
            "SPT.SecType, " \
            "ISNULL(SPT.MarketCapCategory, 'N/A') AS MarketCapCategory, " \
            "PCVH.SecurityID, " \
            "PCVH.DealTermsCash, " \
            "PCVH.DealTermsStock, " \
            "PCVH.DealValue, " \
            "PCVH.DealClosingDate, " \
            "CASE " \
            "WHEN SPT.SecType = 'FxFwd' THEN 0.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Equity Special Situations'  THEN 1.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Opportunistic'  THEN 1.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Merger Arbitrage'  THEN " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') THEN " \
            "CASE PCVH.DealValue WHEN 0 THEN 0.0 ELSE PCVH.DealTermsStock/PCVH.DealValue END " \
            "ELSE 1.0 END " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Credit Opportunities'  THEN " \
            "CASE WHEN SPT.SecType IN ('EQ','ExchOpt') THEN 1.0 ELSE 0.0 END " \
            "ELSE NULL END " \
            "AS [Equity Risk Factor], " \
            "PCVH.DV_01, " \
            "PCVH.CR_01, " \
            "PCVH.Adj_CR_01, " \
            "ISNULL(SPT.UltimateCountry,'N/A') AS UltimateCountry, " \
            "ISNULL(PCVH.AlphaHedge,'NA') AS AlphaHedge, " \
            "SUM(PCVH.ExposureLong_USD) AS Exposure_Long, " \
            "SUM(PCVH.ExposureShort_USD) AS Exposure_Short, " \
            "SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD) AS Exposure_Net, " \
            "CASE WHEN NAV.NAV = 0 THEN NULL " \
            "ELSE 100.0*(SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD)/NAV.NAV) END " \
            "AS [Exposure_Net(%)], " \
            "SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD) AS NetMktVal, " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') " \
            "THEN ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD)) ELSE NULL END AS Capital, " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') " \
            "THEN 100.0*(ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD))/NAV.NAV) ELSE NULL " \
            "END AS [Capital % of NAV], " \
            "100.0*SUM(PCVH.Base_Case_NAV_Impact) AS BaseCaseNavImpact, " \
            "100.0*SUM(PCVH.Outlier_NAV_Impact) AS OutlierNavImpact, " \
            "SUM(PCVH.Qty) AS QTY, " \
            "ISNULL(PCVH.TradeGroupLongShortFlag,'NA') AS LongShort, " \
            "ISNULL(SPT.GICSSectorName,'NA') AS Sector, " \
            "ISNULL(SPT.GICSIndustryName,'NA') AS Industry, " \
            "NAV.NAV " \
            "FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH " \
            "INNER JOIN PnLAppDb.dbo.Funds AS F ON PCVH.Portfolio = F.FundID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON PCVH.SecurityID = SPT.ID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TRDGRP ON PCVH.TradeGroupId = SPT_TRDGRP.ID " \
            "LEFT OUTER JOIN PnLAppDb.pnl.DailyNAV AS NAV ON PCVH.Portfolio = NAV.FundId " \
            "AND PCVH.TradeDate = NAV.[DATE] " \
            "WHERE  TradeDate >= '" + start_date_yyyy_mm_dd + "' AND TradeDate <= '" + end_date_yyyy_mm_dd + "' " \
            + tg_filter_clause + \
            fund_code_filtuer_clause + \
            "GROUP BY " \
            "F.FundName, PCVH.FundCode,PCVH.TradeDate, PCVH.[Marketing GROUP], SPT_TRDGRP.SecName, " \
            "PCVH.TradeGroupId, SPT.Ticker, SPT.BloombergGlobalId, SPT.BloombergGID, " \
            "SPT.SecType, ISNULL(SPT.MarketCapCategory, 'N/A'), " \
            "PCVH.SecurityId, SPT.UltimateCountry, " \
            "ISNULL(PCVH.AlphaHedge,'NA'),PCVH.TradeGroupLongShortFlag, ISNULL(SPT.GICSSectorName,'NA'), " \
            "ISNULL(SPT.GICSIndustryName,'NA'), ISNULL(PCVH.TradeGroupCatalystRating, 'NA'), " \
            "ISNULL(PCVH.[Bucket],'NA'), PCVH.DealTermsCash, PCVH.DealTermsStock, " \
            "PCVH.DealValue, PCVH.DealClosingDate, " \
            "PCVH.DV_01, PCVH.CR_01, PCVH.Adj_CR_01, NAV.NAV, PCVH.Analyst " \
            "ORDER BY PCVH.TradeDate "
    # endregion
    results = optimized_execute(query)
    df = pd.DataFrame(results, columns=["Date", "FundCode", "TradeGroup", "TradeGroup_Tradar_Name", "Sleeve",
                                        "Bucket", "TradeGroupId", "Ticker", "SecType", "MarketCapCategory",
                                        "SecurityId", "DealTermsCash", "DealTermsStock", "DealValue", 
                                        "DealClosingDate", "Equity_Risk_Factor", "DV01", "CR01", "Adj_CR01",
                                        "UltimateCountry", "AlphaHedge", "Exposure_Long", "Exposure_Short",
                                        "Exposure_Net", "Exposure_Net(%)", "NetMktVal", "Capital($)", 
                                        "Capital(%)", "BaseCaseNavImpact", "OutlierNavImpact", "Qty", 
                                        "LongShort", "Sector", "Industry", "Fund_NAV"])

    float_cols = ['NetMktVal', 'Capital($)', 'Capital(%)', 'Exposure_Net', 'Exposure_Net(%)', 'BaseCaseNavImpact']
    df[float_cols] = df[float_cols].astype(float)
    df['Equity_Risk_Exp'] = df['Equity_Risk_Factor'].astype(float)*df['Exposure_Net'].astype(float)
    df[['DV01', 'CR01', 'Adj_CR01']] = df[['DV01', 'CR01', 'Adj_CR01']].fillna(0).astype(float)
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x))
    df["DealClosingDate"] = df["DealClosingDate"].apply(lambda x: pd.to_datetime(x))

    as_of_dt = datetime.datetime.strptime(end_date_yyyy_mm_dd, '%Y-%m-%d')

    def duration_calc(days):
        if days <= 90:
            return '0M-3M'
        if days <= 180:
            return '3M-6M'
        if days <= 365:
            return '6M-12M'
        return 'Yr+'

    df["DealDuration"] = df["DealClosingDate"].apply(lambda x: None if pd.isnull(x) else
                                                     duration_calc((as_of_dt - pd.to_datetime(x)).days))

    # region ADJUSTING FOR POSTED TRADES AFTER TRADE DATE - AFFECTING QTY ONLY
    # adjust for end_date_yyyy_mm_dd - take trades with tradedate = end_date_yyyy_mm_dd and posted_date
    #AFTER end_date_yyyy_mm_dd
    # adjust the Qty into the pcvh_df
    if NP_qty_rollup:
        qty_adjust_query = "SELECT " \
                            "A.TradeDate AS [DATE], " \
                            "CASE A.Fund " \
                            "WHEN 'Columbia' THEN 'CAM' " \
                            "WHEN 'Litman Gregory' THEN 'LG' " \
                            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
                            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
                            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
                            "WHEN 'TransAmerica' THEN 'TAF' " \
                            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
                            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
                            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
                            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'LEV' " \
                            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL COMMINGLED MASTER FUND LP' THEN 'MACO' " \
                            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
                            "END AS FundCode, " \
                            "A.TradeGroupId, " \
                            "A.SecurityId, " \
                            "SUM((CASE SPT.SecType WHEN 'ExchOpt' THEN 100.0 ELSE 1.0 END)*A.Shares) AS Qty " \
                            "FROM " \
                            "[PnLAppDb].[dbo].[vTradesFlatView] AS A " \
                            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON A.SecurityId = SPT.ID " \
                            "WHERE PostedDate >= DATEADD(DAY,1,TradeDate) AND TradeDate >= '" \
                            + start_date_yyyy_mm_dd + "' " \
                            "GROUP BY A.TradeDate, A.SecurityId, A.Ticker, A.Fund, A.TradeGroupId, SPT.SecType "
        results = optimized_execute(qty_adjust_query)
        qty_adjust_df = pd.DataFrame(results, columns=['Date', 'FundCode', 'TradeGroupId', 'SecurityId', 'Qty_adj'])
        qty_adjust_df[['TradeGroupId', 'SecurityId', 'Qty_adj']] = qty_adjust_df[['TradeGroupId',
                                                                                  'SecurityId', 'Qty_adj']].astype(float)
        qty_adjust_df['Date'] = qty_adjust_df['Date'].apply(lambda x: pd.to_datetime(x))

        df[['TradeGroupId', 'SecurityId', 'Qty']] = df[['TradeGroupId', 'SecurityId', 'Qty']].astype(float)
        df = pd.merge(df, qty_adjust_df, how='left', on=['Date', 'FundCode', 'TradeGroupId', 'SecurityId'])
        df['Qty'] = df['Qty'] + df['Qty_adj'].fillna(0)
        del df['Qty_adj']
    # endregion
    print('PCVH retrieval completed....')
    return df

In [9]:
def get_NAV_df_by_date(fund_name = None, start_date_yyyy_mm_dd=None):
    if start_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '20150101'
    elif (start_date_yyyy_mm_dd is not None) & (type(start_date_yyyy_mm_dd) != str):
        start_date_yyyy_mm_dd = start_date_yyyy_mm_dd.strftime('%Y%m%d')

    query = "SELECT A.[DATE], A.NAV, " \
            "CASE B.FundName " \
            "WHEN 'Columbia' THEN 'CAM' " \
            "WHEN 'Litman Gregory' THEN 'LG' " \
            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
            "WHEN 'TransAmerica' THEN 'TAF' " \
            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'LEV' " \
            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL' THEN 'MACO' " \
            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
            "END AS FundCode " \
            "FROM PnLAppDb.pnl.DailyNAV AS A " \
            "INNER JOIN PnLAppDb.dbo.Funds AS B ON A.FundId = B.FundID "
 
    if fund_name is not None and start_date_yyyy_mm_dd is not None:
        query += "WHERE  A.[Date] > '" + start_date_yyyy_mm_dd + "' AND B.FundName = '" + fund_name + "' "
    elif fund_name is not None:
        query += " WHERE B.FundName = '" + fund_name + "' "
    elif start_date_yyyy_mm_dd is not None:
        query += "WHERE  A.[Date] > '" + start_date_yyyy_mm_dd + "' "

    results = optimized_execute(query)
    df = pd.DataFrame(results, columns=["Date", "NAV", "FundCode"])
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df.index.name = 'Date'
    df = df.sort_index()
    df['NAV'] = df['NAV'].astype(float)
    #idx = pd.date_range(df['Date'].min(), df['Date'].max())
    #df = df.reindex(idx, fill_value=np.nan)
    del df['Date']
    df.reset_index(level=0, inplace=True)
    df.rename(columns={"index": "Date"}, inplace=True)
    df.index.name = 'Date'
    df.ffill(inplace=True)
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    # df.reset_index(inplace=True)
    df = df.reset_index(level=0, drop=True).reset_index()  # Added by Kshitij
    del df['index']
    return df[df['Date'] <= (datetime.datetime.today() - datetime.timedelta(days=1))]

In [10]:
def get_tradegroups_total_pnl(fundcode=None, tg=None, start_date_yyyy_mm_dd=None):
    slicer = dfutils.df_slicer()
    if (start_date_yyyy_mm_dd is not None) & (type(start_date_yyyy_mm_dd) != str):
            start_date_yyyy_mm_dd = start_date_yyyy_mm_dd.strftime('%Y%m%d')
    elif start_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '20150101'
                                                                                        
    query = "SELECT `DATE`, Fund, TradeGroup, pnl, cumpnl FROM " + DB_NAME + ".`tradegroups_pnl_cache`"
    if fundcode is not None and tg is not None:
        query += " WHERE Fund = '" + fundcode + "' AND TradeGroup ='" + tg + "' AND `DATE` > " + start_date_yyyy_mm_dd
    elif fundcode is not None:
        query += " WHERE Fund = '" + fundcode + "' AND `DATE` > " + start_date_yyyy_mm_dd
    elif tg is not None:
        query += " WHERE TradeGroup = '" + tg + "' AND `DATE` > " + start_date_yyyy_mm_dd
    else:
        query += " WHERE `DATE` > " + start_date_yyyy_mm_dd
        
    res = wic_optimized_execute(query)
    cols = ['Date', 'Fund', 'TradeGroup', 'Total P&L', 'Cumulative Total P&L']
    df = pd.DataFrame(list(res), columns=cols)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df['Total P&L'] = df['Total P&L'].astype(float)
    df['Cumulative Total P&L'] = df['Cumulative Total P&L'].astype(float)
    return df

In [11]:
def mktval_df(pcvh_df):
    float_cols = ['Alpha Exposure', 'AlphaHedge Exposure', 'Hedge Exposure',
                  'Alpha NetMktVal', 'AlphaHedge NetMktVal', 'Hedge NetMktVal',
                  'Alpha GrossMktVal', 'AlphaHedge GrossMktVal', 'Hedge GrossMktVal',
                  'Alpha GrossExp', 'AlphaHedge GrossExp', 'Hedge GrossExp']
    calc_cols = ['Bet Exposure', 'Bet NetMktVal','Bet GrossMktVal', 'Total NetExposure',
                 'Total NetMktVal', 'Total GrossMktVal']

    if len(pcvh_df) == 0: return pd.DataFrame(columns=["Date"]+float_cols+calc_cols)

    alpha_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Alpha']
    alphahedge_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Alpha Hedge']
    hedge_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Hedge']

    alpha_net_exp = alpha_pcvh[['Date', 'Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'Exposure_Net':'Alpha Exposure'})
    alpha_net_mv = alpha_pcvh[['Date', 'NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'Alpha NetMktVal'})
    alpha_gross_mv = alpha_pcvh[['Date', 'NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'Alpha GrossMktVal'})
    alpha_gross_exp = alpha_pcvh[['Date', 'Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'Alpha GrossExp'})

    alphahedge_net_exp = alphahedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'Exposure_Net':'AlphaHedge Exposure'})
    alphahedge_net_mv = alphahedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'AlphaHedge NetMktVal'})
    alphahedge_gross_mv = alphahedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'AlphaHedge GrossMktVal'})
    alphahedge_gross_exp = alphahedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'AlphaHedge GrossExp'})

    hedge_net_exp = hedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date','Exposure_Net':'Hedge Exposure'})
    hedge_net_mv = hedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'Hedge NetMktVal'})
    hedge_gross_mv = hedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'Hedge GrossMktVal'})
    hedge_gross_exp = hedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'Hedge GrossExp'})

    mktval_df = pd.merge(alpha_net_exp, alphahedge_net_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_net_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_gross_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_gross_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_gross_exp, how='outer', on=['Date']).fillna(0)

    mktval_df[float_cols] = mktval_df[float_cols].astype(float)
    mktval_df['Bet Exposure'] = mktval_df['Alpha Exposure']+mktval_df['AlphaHedge Exposure']
    mktval_df['Bet NetMktVal'] = mktval_df['Alpha NetMktVal']+mktval_df['AlphaHedge NetMktVal']
    mktval_df['Bet GrossMktVal'] = mktval_df['Alpha GrossMktVal']+mktval_df['AlphaHedge GrossMktVal']
    mktval_df['Total NetExposure'] = mktval_df['Alpha Exposure']+mktval_df['AlphaHedge Exposure']+mktval_df['Hedge Exposure']
    mktval_df['Total GrossExposure'] = mktval_df['Alpha GrossExp']+mktval_df['AlphaHedge GrossExp']+mktval_df['Hedge GrossExp']
    mktval_df['Total NetMktVal'] = mktval_df['Alpha NetMktVal']+mktval_df['AlphaHedge NetMktVal']+mktval_df['Hedge NetMktVal']
    mktval_df['Total GrossMktVal'] = mktval_df['Alpha GrossMktVal']+mktval_df['AlphaHedge GrossMktVal']+mktval_df['Hedge GrossMktVal']


    return mktval_df.sort_values(by='Date')

In [12]:
def get_active_tradegroups():
    '''
    Retrieves the Fund and TradeGroup combinations where Status == 'ACTIVE' on the main TradeGroup Performance page
    '''
    slicer = dfutils.df_slicer()
    today = datetime.datetime.today()
    last_date = slicer.prev_n_business_days(1, today)
    last_date_yyyymmdd = last_date.strftime('%Y%m%d')
    
    query = "SELECT `Date`, Fund, TradeGroup FROM " + DB_NAME + ".tradegroup_attribution_to_fund_nav_dollar " \
            "WHERE `Date` = " + last_date_yyyymmdd + " AND Status = 'ACTIVE'"
    
    res = wic_optimized_execute(query)
    cols=['Date', 'Fund', 'TradeGroup']
    return pd.DataFrame(list(res), columns=cols)

In [ ]:
def get_tradegroup_overall_pnl_data(fund, tg):
    '''
    Queries for the P&L data that is saved down when runnning the TradeGroup Performance functions
    '''
    
    query = "SELECT `Date`, Fund, TradeGroup, Daily_PnL_Dollar, Daily_PnL_bps " \
            "FROM wic.tradegroup_overall_pnl " \
            "WHERE Fund = '" + fund + "' AND TradeGroup = '" + tg + "' "
    
    res = wic_optimized_execute(query)
    cols=['Date', 'Fund', 'TradeGroup', 'Total P&L', 'Daily_PnL_bps']
    return pd.DataFrame(list(res), columns=cols)

In [ ]:
PCVH = get_position_calculated_values_history()

In [ ]:
def calculate_max_drawdown_by_tradegroup(PCVH):
    '''
    Calcualtes the Drawdown page information for every TradeGroup in each Fund. Uses the saved P&L data from
    the TradeGroup Performance page.
    '''
    
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    PCVH['Date'] = PCVH['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    #Get only the active TradeGroups
    active_tgs = get_active_tradegroups()
    
    drawdown_df = pd.DataFrame(columns=['Date', 'Fund', 'TradeGroup', 'Last_Date', 'NAV_Max_bps', 'NAV_Min_bps',
                                        'NAV_Last_bps', 'NAV_MaxToMin_Drawdown_bps', 'NAV_MaxToLast_Drawdown_bps',
                                        'NAV_Max_Date', 'NAV_Min_Date', 'ROC_Max_bps', 'ROC_Min_bps',
                                        'ROC_Last_bps', 'ROC_MaxToMin_Drawdown_Pct', 'ROC_MaxToLast_Drawdown_Pct',
                                        'ROC_Max_Date', 'ROC_Min_Date', 'ROMC_Max_bps',
                                        'ROMC_Min_bps', 'ROMC_Last_bps', 'ROMC_MaxToMin_Drawdown_Pct',
                                        'ROMC_MaxToLast_Drawdown_Pct', 'ROMC_Max_Date', 'ROMC_Min_Date'])
    
    for i in range(0,len(active_tgs)):
        fund_code = active_tgs['Fund'][i]
        tg = active_tgs['TradeGroup'][i]
        
        tg_overall_pnl_df = get_tradegroup_overall_pnl_data(fund_code, tg)
        pnl_df = tg_overall_pnl_df.sort_values(by='Date').reset_index(drop=True)
        pnl_df['Date'] = pnl_df['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
        pcvh = PCVH[(PCVH.FundCode == fund_code)&(PCVH.TradeGroup == tg)].copy()
        
        #Calculates the Cumulative Sum of the P&L data (NAV contribution)
        pnl_df['Cum_PnL_NAV_bps'] = pnl_df['Daily_PnL_bps'].cumsum()
        
        #Calculates the Cumulative Sum of the P&L data (ROC and ROMC)
        cap_df = pd.DataFrame()
        if len(pcvh) > 0:
            tg_capital_df = mktval_df(pcvh)
            cap_df = pd.merge(pnl_df, tg_capital_df[['Date', 'Bet GrossMktVal']], how='inner', on=['Date'])
            if len(cap_df) > 1:
                cap_df['Shifted_Capital_ROC'] = cap_df['Bet GrossMktVal'].shift(1)
                if pd.isnull(cap_df['Shifted_Capital_ROC'].iloc[0]):
                    cap_df.loc[0, 'Shifted_Capital_ROC'] = cap_df['Bet GrossMktVal'].iloc[0]
                cap_df['Shifted_Capital_ROC'] = cap_df['Shifted_Capital_ROC'].apply(lambda x: np.nan if x == 0 else x)
                cap_df['Shifted_FF_Capital_ROC'] = cap_df['Shifted_Capital_ROC'].ffill()
                cap_df['PnL_Over_Cap_bps'] = 1e4*(cap_df["Total P&L"]/
                                                cap_df["Shifted_FF_Capital_ROC"]).replace([np.inf, -np.inf], np.nan)

                mean_cap = cap_df['Shifted_FF_Capital_ROC'].fillna(0).astype(float).mean()
                cap_df["Cum_PnL_ROC_bps"] = (1e4*((1.0+(cap_df["PnL_Over_Cap_bps"].astype(float)/1e4)).cumprod()-1))/100
                cap_df['Cum_PnL_ROMC_bps'] = (1e4*(cap_df['Total P&L'].cumsum()/mean_cap))/100
        
        #Calulates the max-min drawdown and the max-curr drawdown for NAV, ROMC, and ROMC
        if ((pnl_df.empty) or('Daily_PnL_bps' not in pnl_df.columns) or (pnl_df['Daily_PnL_bps'].isnull().all())):
            last_NAV = None
            lastdate_NAV = None
            maxret_NAV = None
            maxdate_NAV = None
            minret_NAV = None
            mindate_NAV = None
            curr_drawdown_NAV = None
            max_drawdown_NAV = None
        else:    
            last_valid_index_NAV = pnl_df.Cum_PnL_NAV_bps.last_valid_index()
            last_NAV = pnl_df.Cum_PnL_NAV_bps.iloc[last_valid_index_NAV]
            lastdate_NAV = pnl_df.loc[pnl_df.Cum_PnL_NAV_bps == last_NAV]['Date'].values[0]

            maxret_NAV = pnl_df.Cum_PnL_NAV_bps.max()
            maxdate_NAV = pnl_df.loc[pnl_df.Cum_PnL_NAV_bps == maxret_NAV]['Date'].values[0]

            if maxdate_NAV == lastdate_NAV:
                minret_NAV = None
                mindate_NAV = None
                curr_drawdown_NAV = None
                max_drawdown_NAV = None
            else:
                minret_NAV = pnl_df[pnl_df.Date > maxdate_NAV].Cum_PnL_NAV_bps.min()
                mindate_NAV = pnl_df.loc[pnl_df.Cum_PnL_NAV_bps == minret_NAV]['Date'].values[0]
                curr_drawdown_NAV = (maxret_NAV-last_NAV)
                max_drawdown_NAV = (maxret_NAV-minret_NAV)

        if ((cap_df.empty) or('PnL_Over_Cap_bps' not in cap_df.columns) or (cap_df['PnL_Over_Cap_bps'].isnull().all())):
            last_ROC = None
            lastdate_ROC = None
            maxret_ROC = None
            maxdate_ROC = None
            minret_ROC = None
            mindate_ROC = None
            curr_drawdown_ROC = None
            max_drawdown_ROC = None
            last_ROMC = None
            lastdate_ROMC = None
            maxret_ROMC = None
            maxdate_ROMC = None
            minret_ROMC = None
            mindate_ROMC = None
            curr_drawdown_ROMC = None
            max_drawdown_ROMC = None
        else: 
            last_valid_index_ROC = cap_df.Cum_PnL_ROC_bps.last_valid_index()
            last_ROC = cap_df.Cum_PnL_ROC_bps.iloc[last_valid_index_ROC]
            lastdate_ROC = cap_df.loc[cap_df.Cum_PnL_ROC_bps == last_ROC]['Date'].values[0]
            maxret_ROC = cap_df.Cum_PnL_ROC_bps.max()
            maxdate_ROC = cap_df.loc[cap_df.Cum_PnL_ROC_bps == maxret_ROC]['Date'].values[0]

            if maxdate_ROC == lastdate_ROC:
                minret_ROC = None
                mindate_ROC = None
                curr_drawdown_ROC = None
                max_drawdown_ROC = None
            else:
                minret_ROC = cap_df[cap_df.Date > maxdate_ROC].Cum_PnL_ROC_bps.min()
                mindate_ROC = cap_df.loc[cap_df.Cum_PnL_ROC_bps == minret_ROC]['Date'].values[0]
                curr_drawdown_ROC = (((1+(maxret_ROC/100))-(1+(last_ROC/100)))/(1+(maxret_ROC/100)))*100
                max_drawdown_ROC = (((1+(maxret_ROC/100))-(1+(minret_ROC/100)))/(1+(maxret_ROC/100)))*100

            last_valid_index_ROMC = cap_df.Cum_PnL_ROMC_bps.last_valid_index()
            last_ROMC = cap_df.Cum_PnL_ROMC_bps.iloc[last_valid_index_ROMC]
            lastdate_ROMC = cap_df.loc[cap_df.Cum_PnL_ROMC_bps == last_ROMC]['Date'].values[0]
            maxret_ROMC = cap_df.Cum_PnL_ROMC_bps.max()
            maxdate_ROMC = cap_df.loc[cap_df.Cum_PnL_ROMC_bps == maxret_ROMC]['Date'].values[0]

            if maxdate_ROMC == lastdate_ROMC:
                minret_ROMC = None
                mindate_ROMC = None
                curr_drawdown_ROMC = None
                max_drawdown_ROMC = None
            else:
                minret_ROMC = cap_df[cap_df.Date > maxdate_ROMC].Cum_PnL_ROMC_bps.min()
                mindate_ROMC = cap_df.loc[cap_df.Cum_PnL_ROMC_bps == minret_ROMC]['Date'].values[0]
                curr_drawdown_ROMC = (((1+(maxret_ROMC/100))-(1+(last_ROMC/100)))/(1+(maxret_ROMC/100))*100)
                max_drawdown_ROMC = (((1+(maxret_ROMC/100))-(1+(minret_ROMC/100)))/(1+(maxret_ROMC/100))*100)

        #Returns the results and the corresponding dates as a pandas series for simplicity right now
        drawdown_df.loc[i, 'Date'] = today
        drawdown_df.loc[i, 'Fund'] = fund_code
        drawdown_df.loc[i, 'TradeGroup'] = tg
        drawdown_df.loc[i, 'Last_Date'] = None if lastdate_NAV is None else pd.to_datetime(str(lastdate_NAV)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'NAV_Max_bps'] = maxret_NAV
        drawdown_df.loc[i, 'NAV_Last_bps'] = last_NAV
        drawdown_df.loc[i, 'NAV_Min_bps'] = minret_NAV
        drawdown_df.loc[i, 'NAV_MaxToMin_Drawdown_bps'] = max_drawdown_NAV
        drawdown_df.loc[i, 'NAV_MaxToLast_Drawdown_bps'] = curr_drawdown_NAV
        drawdown_df.loc[i, 'NAV_Max_Date'] = None if maxdate_NAV is None else pd.to_datetime(str(maxdate_NAV)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'NAV_Min_Date'] = None if mindate_NAV is None else pd.to_datetime(str(mindate_NAV)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROC_Max_bps'] = maxret_ROC
        drawdown_df.loc[i, 'ROC_Last_bps'] = last_ROC
        drawdown_df.loc[i, 'ROC_Min_bps'] = minret_ROC
        drawdown_df.loc[i, 'ROC_MaxToMin_Drawdown_Pct'] = max_drawdown_ROC
        drawdown_df.loc[i, 'ROC_MaxToLast_Drawdown_Pct'] = curr_drawdown_ROC 
        drawdown_df.loc[i, 'ROC_Max_Date'] = None if maxdate_ROC is None else pd.to_datetime(str(maxdate_ROC)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROC_Min_Date'] = None if mindate_ROC is None else pd.to_datetime(str(mindate_ROC)).strftime('%Y-%m-%d')
        #drawdown_df.loc[i, 'ROC_Last_Date'] = None if lastdate_ROC is None else pd.to_datetime(str(lastdate_ROC)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROMC_Max_bps'] = maxret_ROMC
        drawdown_df.loc[i, 'ROMC_Last_bps'] = last_ROMC
        drawdown_df.loc[i, 'ROMC_Min_bps'] = minret_ROMC
        drawdown_df.loc[i, 'ROMC_MaxToMin_Drawdown_Pct'] = max_drawdown_ROMC
        drawdown_df.loc[i, 'ROMC_MaxToLast_Drawdown_Pct'] = curr_drawdown_ROMC
        drawdown_df.loc[i, 'ROMC_Max_Date'] = None if maxdate_ROMC is None else pd.to_datetime(str(maxdate_ROMC)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROMC_Min_Date'] = None if mindate_ROMC is None else pd.to_datetime(str(mindate_ROMC)).strftime('%Y-%m-%d')
        #drawdown_df.loc[i, 'ROMC_Last_Date'] = None if lastdate_ROMC is None else pd.to_datetime(str(lastdate_ROMC)).strftime('%Y-%m-%d')

    #con = engine.connect()
    #drawdown_df.to_sql(con=con, schema='wic', name='risk_tradegroup_drawdown', if_exists='append', chunksize=1000, index=False)
    #con.close()    
        
    return drawdown_df

In [ ]:
x = calculate_max_drawdown_by_tradegroup(PCVH)

In [ ]:
#Everything up until this point is what the portal uses to run

In [14]:
#This can be used if you want to complete the drawdowns for a single tradegroup in a particular fund
def calculate_max_drawdown_individual(fund_code, tg):
    '''
    Calculates the Drawdown for a single TradeGroup and Fund combination.
    ''' 
    #Collects all the data to perform the calcs
    tg_daily_pnl = get_tradegroups_total_pnl(fundcode=fund_code, tg = tg) #Daily dollar P&L
    fund_nav_df = get_NAV_df_by_date() #Fund NAV
    fund_nav_df = fund_nav_df[fund_nav_df.FundCode == fund_code]
    pcvh = get_position_calculated_values_history(limit_to_tradegroups=[tg], fund_code=fund_code) #Used to calculate over own cap values

    #Takes the daily tradegroup dollar P&L and converts it to bps and then calculates the cumulative sum -- NAV
    pnl_df = pd.merge(tg_daily_pnl, fund_nav_df[['Date', 'NAV']], how='inner', on=['Date'])
    pnl_df['Shifted_Capital_NAV'] = pnl_df['NAV'].shift(1)
    if pd.isnull(pnl_df['Shifted_Capital_NAV'].iloc[0]):
        pnl_df.loc[0, 'Shifted_Capital_NAV'] = pnl_df['NAV'].iloc[0]
    pnl_df['Shifted_Capital_NAV'] = pnl_df['Shifted_Capital_NAV'].apply(lambda x: np.nan if x == 0 else x)
    pnl_df['Shifted_FF_Capital_NAV'] = pnl_df['Shifted_Capital_NAV'].ffill()
    pnl_df['Daily_PnL_bps'] = 1e4*(pnl_df["Total P&L"]/pnl_df["Shifted_FF_Capital_NAV"]).replace([np.inf, -np.inf], np.nan)
    pnl_df['Cum_PnL_NAV_bps'] = pnl_df['Daily_PnL_bps'].cumsum()

    #Takes the daily tradegroup dollar P&L and calculates ROC and ROMC (bps)
    roc_df = pd.DataFrame()
    if len(pcvh) > 0:
        tg_capital_df = mktval_df(pcvh)
        roc_df = pd.merge(tg_daily_pnl, tg_capital_df[['Date', 'Bet GrossMktVal']], how='inner', on=['Date'])
        if len(roc_df) > 1:
            roc_df['Shifted_Capital_ROC'] = roc_df['Bet GrossMktVal'].shift(1)
            if pd.isnull(roc_df['Shifted_Capital_ROC'].iloc[0]):
                roc_df.loc[0, 'Shifted_Capital_ROC'] = roc_df['Bet GrossMktVal'].iloc[0]
            roc_df['Shifted_Capital_ROC'] = roc_df['Shifted_Capital_ROC'].apply(lambda x: np.nan if x == 0 else x)
            roc_df['Shifted_FF_Capital_ROC'] = roc_df['Shifted_Capital_ROC'].ffill()
            roc_df['PnL_Over_Cap_bps'] = 1e4*(roc_df["Total P&L"]/
                                            roc_df["Shifted_FF_Capital_ROC"]).replace([np.inf, -np.inf], np.nan)

            mean_cap = roc_df['Shifted_FF_Capital_ROC'].fillna(0).astype(float).mean()
            roc_df["Cum_PnL_ROC_bps"] = (1e4*((1.0+(roc_df["PnL_Over_Cap_bps"].astype(float)/1e4)).cumprod()-1))/100
            roc_df['Cum_PnL_ROMC_bps'] = (1e4*(roc_df['Total P&L'].cumsum()/mean_cap))/100

    combined_df = pd.merge(pnl_df, roc_df, how='outer', on=['Date', 'Fund', "TradeGroup", 'Total P&L', 'Cumulative Total P&L'])
    print(combined_df)
    
    #Calulates the max-min drawdown and the max-curr drawdown for NAV, ROMC, and ROMC
    last_valid_index_NAV = combined_df.Cum_PnL_NAV_bps.last_valid_index()
    last_NAV = combined_df.Cum_PnL_NAV_bps.iloc[last_valid_index_NAV]
    lastdate_NAV = combined_df.loc[combined_df.Cum_PnL_NAV_bps == last_NAV]['Date'].values[0]

    maxret_NAV = combined_df.Cum_PnL_NAV_bps.max()
    maxdate_NAV = combined_df.loc[combined_df.Cum_PnL_NAV_bps == maxret_NAV]['Date'].values[0]

    if maxdate_NAV == lastdate_NAV:
        minret_NAV = None
        mindate_NAV = None
        curr_drawdown_NAV = None
        max_drawdown_NAV = None
    else:
        minret_NAV = combined_df[combined_df.Date > maxdate_NAV].Cum_PnL_NAV_bps.min()
        mindate_NAV = combined_df.loc[combined_df.Cum_PnL_NAV_bps == minret_NAV]['Date'].values[0]
        curr_drawdown_NAV = (maxret_NAV-last_NAV)
        max_drawdown_NAV = (maxret_NAV-minret_NAV)

    if combined_df.PnL_Over_Cap_bps.isnull().values.all():
        last_ROC = None
        lastdate_ROC = None
        maxret_ROC = None
        maxdate_ROC = None
        minret_ROC = None
        mindate_ROC = None
        curr_drawdown_ROC = None
        max_drawdown_ROC = None
        last_ROMC = None
        lastdate_ROMC = None
        maxret_ROMC = None
        maxdate_ROMC = None
        minret_ROMC = None
        mindate_ROMC = None
        curr_drawdown_ROMC = None
        max_drawdown_ROMC = None
    else: 
        last_valid_index_ROC = combined_df.Cum_PnL_ROC_bps.last_valid_index()
        last_ROC = combined_df.Cum_PnL_ROC_bps.iloc[last_valid_index_ROC]
        lastdate_ROC = combined_df.loc[combined_df.Cum_PnL_ROC_bps == last_ROC]['Date'].values[0]
        maxret_ROC = combined_df.Cum_PnL_ROC_bps.max()
        maxdate_ROC = combined_df.loc[combined_df.Cum_PnL_ROC_bps == maxret_ROC]['Date'].values[0]

        if maxdate_ROC == lastdate_ROC:
            minret_ROC = None
            mindate_ROC = None
            curr_drawdown_ROC = None
            max_drawdown_ROC = None
        else:
            minret_ROC = combined_df[combined_df.Date > maxdate_ROC].Cum_PnL_ROC_bps.min()
            mindate_ROC = combined_df.loc[combined_df.Cum_PnL_ROC_bps == minret_ROC]['Date'].values[0]
            curr_drawdown_ROC = (((1+(maxret_ROC/100))-(1+(last_ROC/100)))/(1+(maxret_ROC/100)))*100
            max_drawdown_ROC = (((1+(maxret_ROC/100))-(1+(minret_ROC/100)))/(1+(maxret_ROC/100)))*100


        last_valid_index_ROMC = combined_df.Cum_PnL_ROMC_bps.last_valid_index()
        last_ROMC = combined_df.Cum_PnL_ROMC_bps.iloc[last_valid_index_ROMC]
        lastdate_ROMC = combined_df.loc[combined_df.Cum_PnL_ROMC_bps == last_ROMC]['Date'].values[0]
        maxret_ROMC = combined_df.Cum_PnL_ROMC_bps.max()
        maxdate_ROMC = combined_df.loc[combined_df.Cum_PnL_ROMC_bps == maxret_ROMC]['Date'].values[0]

        if maxdate_ROMC == lastdate_ROMC:
            minret_ROMC = None
            mindate_ROMC = None
            curr_drawdown_ROMC = None
            max_drawdown_ROMC = None
        else:
            minret_ROMC = combined_df[combined_df.Date > maxdate_ROMC].Cum_PnL_ROMC_bps.min()
            mindate_ROMC = combined_df.loc[combined_df.Cum_PnL_ROMC_bps == minret_ROMC]['Date'].values[0]
            curr_drawdown_ROMC = (((1+(maxret_ROMC/100))-(1+(last_ROMC/100)))/(1+(maxret_ROMC/100))*100)
            max_drawdown_ROMC = (((1+(maxret_ROMC/100))-(1+(minret_ROMC/100)))/(1+(maxret_ROMC/100))*100)

    drawdown_df = pd.Series()
    #Returns the results and the corresponding dates as a pandas series for simplicity right now
    drawdown_df['NAV max (bps)'] = maxret_NAV
    drawdown_df['NAV last (bps)'] = last_NAV
    drawdown_df['NAV min (bps)'] = minret_NAV
    drawdown_df['NAV MaxToMin Drawdown bps'] = max_drawdown_NAV/100
    drawdown_df['NAV MaxToLast Drawdown bps'] = curr_drawdown_NAV/100
    drawdown_df['NAV Max Date'] = None if maxdate_NAV is None else pd.to_datetime(str(maxdate_NAV)).strftime('%Y-%m-%d')
    drawdown_df['NAV Min Date'] = None if mindate_NAV is None else pd.to_datetime(str(mindate_NAV)).strftime('%Y-%m-%d')
    drawdown_df['NAV Last Date)'] = None if lastdate_NAV is None else pd.to_datetime(str(lastdate_NAV)).strftime('%Y-%m-%d')
    drawdown_df['ROC max (bps)'] = maxret_ROC
    drawdown_df['ROC last (bps)'] = last_ROC
    drawdown_df['ROC min (bps)'] = minret_ROC
    drawdown_df['ROC MaxToMin Drawdown (%)'] = max_drawdown_ROC
    drawdown_df['ROC MaxToLast Drawdown (%)'] = curr_drawdown_ROC 
    drawdown_df['ROC Max Date'] = None if maxdate_ROC is None else pd.to_datetime(str(maxdate_ROC)).strftime('%Y-%m-%d')
    drawdown_df['ROC Min Date'] = None if mindate_ROC is None else pd.to_datetime(str(mindate_ROC)).strftime('%Y-%m-%d')
    drawdown_df['ROC Last Date'] = None if lastdate_ROC is None else pd.to_datetime(str(lastdate_ROC)).strftime('%Y-%m-%d')
    drawdown_df['ROMC max (bps)'] = maxret_ROMC
    drawdown_df['ROMC last (bps)'] = last_ROMC
    drawdown_df['ROMC min (bps)'] = minret_ROMC
    drawdown_df['ROMC MaxToMin Drawdown (%)'] = max_drawdown_ROMC
    drawdown_df['ROMC MaxToLast Drawdown (%)'] = curr_drawdown_ROMC
    drawdown_df['ROMC Max Date'] = None if maxdate_ROMC is None else pd.to_datetime(str(maxdate_ROMC)).strftime('%Y-%m-%d')
    drawdown_df['ROMC Min Date'] = None if mindate_ROMC is None else pd.to_datetime(str(mindate_ROMC)).strftime('%Y-%m-%d')
    drawdown_df['ROMC Last Date'] = None if lastdate_ROMC is None else pd.to_datetime(str(lastdate_ROMC)).strftime('%Y-%m-%d')

    return combined_df, drawdown_df

In [13]:
fund_code = 'ARB'
tg = "MRT - OHI"
data, res = calculate_max_drawdown_individual(fund_code, tg)

PCVH retrieval completed....
         Date Fund TradeGroup   Total P&L  Cumulative Total P&L           NAV  \
0  2019-01-02  ARB  MRT - OHI -15133.7261                   1.0  1.740943e+09   
1  2019-01-03  ARB  MRT - OHI  -6649.2409                   1.0  1.742339e+09   
2  2019-01-04  ARB  MRT - OHI -10898.4700                   1.0  1.737736e+09   
3  2019-01-07  ARB  MRT - OHI  22074.5856                   3.0  1.739449e+09   
4  2019-01-08  ARB  MRT - OHI -11514.1423                   1.0  1.740441e+09   
5  2019-01-09  ARB  MRT - OHI  17507.8438                   1.0  1.741617e+09   
6  2019-01-10  ARB  MRT - OHI  -4108.0340                   1.0  1.741383e+09   
7  2019-01-11  ARB  MRT - OHI  -2158.9704                   1.0  1.741290e+09   
8  2019-01-14  ARB  MRT - OHI   1050.8300                   3.0  1.742520e+09   
9  2019-01-15  ARB  MRT - OHI  20386.5394                   1.0  1.740819e+09   
10 2019-01-16  ARB  MRT - OHI  10371.6600                   1.0  1.754325e+09   

In [19]:
#This uses all the queries to gather the data instead of the pnl data we save down every night
def calculate_max_drawdown_gather_data(fund_tgs):
    '''
    Calcualtes the Drawdown page information for every TradeGroup in each Fund. Queries Tradar and NP for all
    of the P&L and Capital data.
    '''
    
    start = time.time()
    
    drawdown_df = pd.DataFrame(columns=['Fund', 'TradeGroup', 'NAV max (bps)', 'NAV min (bps)', 'NAV last (bps)',
                                        'NAV MaxToMin Drawdown (%)', 'NAV MaxToLast Drawdown (%)', 'NAV Max Date (bps)',
                                        'NAV Min Date (bps)', 'NAV Last Date (bps)', 'ROC max (bps)', 'ROC min (bps)',
                                        'ROC last (bps)', 'ROC MaxToMin Drawdown (%)', 'ROC MaxToLast Drawdown (%)',
                                        'ROC Max Date (bps)', 'ROC Min Date (bps)', 'ROC Last Date (bps)', 'ROMC max (bps)',
                                        'ROMC min (bps)', 'ROMC last (bps)', 'ROMC MaxToMin Drawdown (%)',
                                        'ROMC MaxToLast Drawdown (%)', 'ROMC Max Date (bps)', 'ROMC Min Date (bps)',
                                        'ROMC Last Date (bps)'])
    
    for i in range(0,len(fund_tgs)):
        fund_code = fund_tgs['Fund'][i]
        tg = fund_tgs['TradeGroup'][i]
        
        #Collects all the data to perform the calcs
        tg_daily_pnl = get_tradegroups_total_pnl(fundcode=fund_code, tg = tg) #Daily dollar P&L
        fund_nav_df = get_NAV_df_by_date() #Fund NAV
        fund_nav_df = fund_nav_df[fund_nav_df.FundCode == fund_code]
        pcvh = get_position_calculated_values_history(limit_to_tradegroups=[tg], fund_code=fund_code) #Used to calculate over own cap values

        #Takes the daily tradegroup dollar P&L and converts it to bps and then calculates the cumulative sum -- NAV
        pnl_df = pd.merge(tg_daily_pnl, fund_nav_df[['Date', 'NAV']], how='inner', on=['Date'])
        pnl_df['Shifted_Capital_NAV'] = pnl_df['NAV'].shift(1)
        if pd.isnull(pnl_df['Shifted_Capital_NAV'].iloc[0]):
            pnl_df.loc[0, 'Shifted_Capital_NAV'] = pnl_df['NAV'].iloc[0]
        pnl_df['Shifted_Capital_NAV'] = pnl_df['Shifted_Capital_NAV'].apply(lambda x: np.nan if x == 0 else x)
        pnl_df['Shifted_FF_Capital_NAV'] = pnl_df['Shifted_Capital_NAV'].ffill()
        pnl_df['Daily_PnL_bps'] = 1e4*(pnl_df["Total P&L"]/pnl_df["Shifted_FF_Capital_NAV"]).replace([np.inf, -np.inf], np.nan)
        pnl_df['Cum_PnL_NAV_bps'] = pnl_df['Daily_PnL_bps'].cumsum()

        #Takes the daily tradegroup dollar P&L and calculates ROC and ROMC (bps)
        roc_df = pd.DataFrame()
        if len(pcvh) > 0:
            tg_capital_df = mktval_df(pcvh)
            roc_df = pd.merge(tg_daily_pnl, tg_capital_df[['Date', 'Bet GrossMktVal']], how='inner', on=['Date'])
            if len(roc_df) > 1:
                roc_df['Shifted_Capital_ROC'] = roc_df['Bet GrossMktVal'].shift(1)
                if pd.isnull(roc_df['Shifted_Capital_ROC'].iloc[0]):
                    roc_df.loc[0, 'Shifted_Capital_ROC'] = roc_df['Bet GrossMktVal'].iloc[0]
                roc_df['Shifted_Capital_ROC'] = roc_df['Shifted_Capital_ROC'].apply(lambda x: np.nan if x == 0 else x)
                roc_df['Shifted_FF_Capital_ROC'] = roc_df['Shifted_Capital_ROC'].ffill()
                roc_df['PnL_Over_Cap_bps'] = 1e4*(roc_df["Total P&L"]/
                                                roc_df["Shifted_FF_Capital_ROC"]).replace([np.inf, -np.inf], np.nan)

                mean_cap = roc_df['Shifted_FF_Capital_ROC'].fillna(0).astype(float).mean()
                roc_df["Cum_PnL_ROC_bps"] = (1e4*((1.0+(roc_df["PnL_Over_Cap_bps"].astype(float)/1e4)).cumprod()-1))/100
                roc_df['Cum_PnL_ROMC_bps'] = (1e4*(roc_df['Total P&L'].cumsum()/mean_cap))/100
        
        combined_df = pd.merge(pnl_df, roc_df, how='outer', on=['Date', 'Fund', "TradeGroup", 'Total P&L', 'Cumulative Total P&L'])

        #Calulates the max-min drawdown and the max-curr drawdown for NAV, ROMC, and ROMC
        
        last_valid_index_NAV = combined_df.Cum_PnL_NAV_bps.last_valid_index()
        last_NAV = combined_df.Cum_PnL_NAV_bps.iloc[last_valid_index_NAV]
        lastdate_NAV = combined_df.loc[combined_df.Cum_PnL_NAV_bps == last_NAV]['Date'].values[0]

        maxret_NAV = combined_df.Cum_PnL_NAV_bps.max()
        maxdate_NAV = combined_df.loc[combined_df.Cum_PnL_NAV_bps == maxret_NAV]['Date'].values[0]

        if maxdate_NAV == lastdate_NAV:
            minret_NAV = None
            mindate_NAV = None
            curr_drawdown_NAV = None
            max_drawdown_NAV = None
        else:
            minret_NAV = combined_df[combined_df.Date > maxdate_NAV].Cum_PnL_NAV_bps.min()
            mindate_NAV = combined_df.loc[combined_df.Cum_PnL_NAV_bps == minret_NAV]['Date'].values[0]
            curr_drawdown_NAV = (maxret_NAV-last_NAV)
            max_drawdown_NAV = (maxret_NAV-minret_NAV)

        if combined_df.PnL_Over_Cap_bps.isnull().values.all():
            last_ROC = None
            lastdate_ROC = None
            maxret_ROC = None
            maxdate_ROC = None
            minret_ROC = None
            mindate_ROC = None
            curr_drawdown_ROC = None
            max_drawdown_ROC = None
            last_ROMC = None
            lastdate_ROMC = None
            maxret_ROMC = None
            maxdate_ROMC = None
            minret_ROMC = None
            mindate_ROMC = None
            curr_drawdown_ROMC = None
            max_drawdown_ROMC = None
        else: 
            last_valid_index_ROC = combined_df.Cum_PnL_ROC_bps.last_valid_index()
            last_ROC = combined_df.Cum_PnL_ROC_bps.iloc[last_valid_index_ROC]
            lastdate_ROC = combined_df.loc[combined_df.Cum_PnL_ROC_bps == last_ROC]['Date'].values[0]
            maxret_ROC = combined_df.Cum_PnL_ROC_bps.max()
            maxdate_ROC = combined_df.loc[combined_df.Cum_PnL_ROC_bps == maxret_ROC]['Date'].values[0]

            if maxdate_ROC == lastdate_ROC:
                minret_ROC = None
                mindate_ROC = None
                curr_drawdown_ROC = None
                max_drawdown_ROC = None
            else:
                minret_ROC = combined_df[combined_df.Date > maxdate_ROC].Cum_PnL_ROC_bps.min()
                mindate_ROC = combined_df.loc[combined_df.Cum_PnL_ROC_bps == minret_ROC]['Date'].values[0]
                curr_drawdown_ROC = (((1+(maxret_ROC/100))-(1+(last_ROC/100)))/(1+(maxret_ROC/100)))*100
                max_drawdown_ROC = (((1+(maxret_ROC/100))-(1+(minret_ROC/100)))/(1+(maxret_ROC/100)))*100


            last_valid_index_ROMC = combined_df.Cum_PnL_ROMC_bps.last_valid_index()
            last_ROMC = combined_df.Cum_PnL_ROMC_bps.iloc[last_valid_index_ROMC]
            lastdate_ROMC = combined_df.loc[combined_df.Cum_PnL_ROMC_bps == last_ROMC]['Date'].values[0]
            maxret_ROMC = combined_df.Cum_PnL_ROMC_bps.max()
            maxdate_ROMC = combined_df.loc[combined_df.Cum_PnL_ROMC_bps == maxret_ROMC]['Date'].values[0]

            if maxdate_ROMC == lastdate_ROMC:
                minret_ROMC = None
                mindate_ROMC = None
                curr_drawdown_ROMC = None
                max_drawdown_ROMC = None
            else:
                minret_ROMC = combined_df[combined_df.Date > maxdate_ROMC].Cum_PnL_ROMC_bps.min()
                mindate_ROMC = combined_df.loc[combined_df.Cum_PnL_ROMC_bps == minret_ROMC]['Date'].values[0]
                curr_drawdown_ROMC = (((1+(maxret_ROMC/100))-(1+(last_ROMC/100)))/(1+(maxret_ROMC/100))*100)
                max_drawdown_ROMC = (((1+(maxret_ROMC/100))-(1+(minret_ROMC/100)))/(1+(maxret_ROMC/100))*100)


        #Returns the results and the corresponding dates as a pandas series for simplicity right now
        drawdown_df.loc[i, 'Fund'] = fund_code
        drawdown_df.loc[i, 'TradeGroup'] = tg
        drawdown_df.loc[i, 'NAV max (bps)'] = maxret_NAV
        drawdown_df.loc[i, 'NAV last (bps)'] = last_NAV
        drawdown_df.loc[i, 'NAV min (bps)'] = minret_NAV
        drawdown_df.loc[i, 'NAV MaxToMin Drawdown (%)'] = max_drawdown_NAV
        drawdown_df.loc[i, 'NAV MaxToLast Drawdown (%)'] = curr_drawdown_NAV
        drawdown_df.loc[i, 'NAV Max Date (bps)'] = None if maxdate_NAV is None else pd.to_datetime(str(maxdate_NAV)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'NAV Min Date (bps)'] = None if mindate_NAV is None else pd.to_datetime(str(mindate_NAV)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'NAV Last Date (bps)'] = None if lastdate_NAV is None else pd.to_datetime(str(lastdate_NAV)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROC max (bps)'] = maxret_ROC
        drawdown_df.loc[i, 'ROC last (bps)'] = last_ROC
        drawdown_df.loc[i, 'ROC min (bps)'] = minret_ROC
        drawdown_df.loc[i, 'ROC MaxToMin Drawdown (%)'] = max_drawdown_ROC
        drawdown_df.loc[i, 'ROC MaxToLast Drawdown (%)'] = curr_drawdown_ROC 
        drawdown_df.loc[i, 'ROC Max Date (bps)'] = None if maxdate_ROC is None else pd.to_datetime(str(maxdate_ROC)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROC Min Date (bps)'] = None if mindate_ROC is None else pd.to_datetime(str(mindate_ROC)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROC Last Date (bps)'] = None if lastdate_ROC is None else pd.to_datetime(str(lastdate_ROC)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROMC max (bps)'] = maxret_ROMC
        drawdown_df.loc[i, 'ROMC last (bps)'] = last_ROMC
        drawdown_df.loc[i, 'ROMC min (bps)'] = minret_ROMC
        drawdown_df.loc[i, 'ROMC MaxToMin Drawdown (%)'] = max_drawdown_ROMC
        drawdown_df.loc[i, 'ROMC MaxToLast Drawdown (%)'] = curr_drawdown_ROMC
        drawdown_df.loc[i, 'ROMC Max Date (bps)'] = None if maxdate_ROMC is None else pd.to_datetime(str(maxdate_ROMC)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROMC Min Date (bps)'] = None if mindate_ROMC is None else pd.to_datetime(str(mindate_ROMC)).strftime('%Y-%m-%d')
        drawdown_df.loc[i, 'ROMC Last Date (bps)'] = None if lastdate_ROMC is None else pd.to_datetime(str(lastdate_ROMC)).strftime('%Y-%m-%d')

    
    print(time.time()-start)
    return drawdown_df

In [22]:
f = get_active_tradegroups().head(20)

In [23]:
y = calculate_max_drawdown(f)
y

NIHD CONV - TRADE
PCVH retrieval completed....
SERV R/R SHORT 2
PCVH retrieval completed....
DOV R/R SHORT
PCVH retrieval completed....
MLNX - NVDA
PCVH retrieval completed....
ISAT LN - APAX & CONSORTIUM
PCVH retrieval completed....
SPY MARKET HEDGE
PCVH retrieval completed....
PACB - ILMN
PCVH retrieval completed....
DXC R/R
PCVH retrieval completed....
CELG - BMY
PCVH retrieval completed....
CREDIT MACRO HEDGES
PCVH retrieval completed....
DOV R/R SHORT
PCVH retrieval completed....
DOV R/R SHORT
PCVH retrieval completed....
SERV R/R SHORT 2
PCVH retrieval completed....
SERV R/R SHORT 2
PCVH retrieval completed....
ALV R/R SHORT
PCVH retrieval completed....
PACB - ILMN
PCVH retrieval completed....
CNDT R/R
PCVH retrieval completed....
EFII - SIRIS CAPITAL
PCVH retrieval completed....
CELG - BMY
PCVH retrieval completed....
LOGM R/R
PCVH retrieval completed....
148.105999947


,Fund,TradeGroup,NAV max (bps),NAV min (bps),NAV last (bps),NAV MaxToMin Drawdown (%),NAV MaxToLast Drawdown (%),NAV Max Date (bps),NAV Min Date (bps),NAV Last Date (bps),ROC max (bps),ROC min (bps),ROC last (bps),ROC MaxToMin Drawdown (%),ROC MaxToLast Drawdown (%),ROC Max Date (bps),ROC Min Date (bps),ROC Last Date (bps),ROMC max (bps),ROMC min (bps),ROMC last (bps),ROMC MaxToMin Drawdown (%),ROMC MaxToLast Drawdown (%),ROMC Max Date (bps),ROMC Min Date (bps),ROMC Last Date (bps)
0,WED,NIHD CONV - TRADE,1021.16,-1237.53,-1144.61,2258.7,2165.78,2018-11-09,2019-04-15,2019-05-09,135.579,-52.2367,-48.8419,79.7252,78.2841,2018-08-01,2019-04-15,2019-05-09,88.5632,-95.2495,-88.6206,97.4807,93.9652,2018-11-09,2019-04-15,2019-05-09
1,LG,SERV R/R SHORT 2,0.14103,-17.3541,-17.3541,17.4951,17.4951,2019-02-25,2019-05-09,2019-05-09,0.392044,-20.8432,-20.8432,21.1523,21.1523,2019-02-25,2019-05-09,2019-05-09,0.159701,-19.6778,-19.6778,19.8059,19.8059,2019-02-25,2019-05-09,2019-05-09
2,LG,DOV R/R SHORT,5.41755,-11.612,-10.797,17.0295,16.2145,2018-12-26,2019-05-07,2019-05-09,7.86688,-11.5335,-11.0568,17.9855,17.5436,2018-12-26,2019-05-07,2019-05-09,6.4792,-13.2371,-12.307,18.5166,17.6431,2018-12-26,2019-05-07,2019-05-09
3,ARB,MLNX - NVDA,4.08812,-2.58713,-2.58713,6.67525,6.67525,2019-04-24,2019-05-09,2019-05-09,1.46182,-0.921316,-0.921316,2.34881,2.34881,2019-04-24,2019-05-09,2019-05-09,1.80642,-1.16626,-1.16626,2.91993,2.91993,2019-04-24,2019-05-09,2019-05-09
4,ARB,ISAT LN - APAX & CONSORTIUM,0.0828686,-1.70266,-1.60281,1.78553,1.68568,2019-03-25,2019-05-08,2019-05-09,0.33019,-4.47026,-4.27372,4.78465,4.58876,2019-03-25,2019-05-08,2019-05-09,0.22414,-4.59285,-4.32294,4.80622,4.53691,2019-03-25,2019-05-08,2019-05-09
5,WED,SPY MARKET HEDGE,47.1214,-898.815,-714.742,945.936,761.863,2018-05-03,2019-05-03,2019-05-09,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,ARB,PACB - ILMN,1.02528,-4.54648,-2.34698,5.57176,3.37226,2018-11-30,2019-01-30,2019-05-09,3.67982,-9.68802,-2.96043,12.8934,6.40458,2018-11-30,2019-01-30,2019-05-09,2.0106,-9.1223,-4.70024,10.9135,6.57857,2018-11-30,2019-01-30,2019-05-09
7,LG,DXC R/R,12.6501,3.21215,3.21215,9.43798,9.43798,2019-04-29,2019-05-09,2019-05-09,16.4646,6.01317,6.01317,8.97395,8.97395,2019-04-29,2019-05-09,2019-05-09,11.822,3.69109,3.69109,7.27131,7.27131,2019-04-29,2019-05-09,2019-05-09
8,ARB,CELG - BMY,1.87312,-2.09974,-1.38937,3.97287,3.26249,2019-02-01,2019-02-28,2019-05-09,2.50614,-3.61619,-0.292173,5.97264,2.72989,2019-01-09,2019-02-28,2019-05-09,2.06525,-2.2731,-1.49115,4.25057,3.48445,2019-02-01,2019-02-28,2019-05-09
9,TACO,CREDIT MACRO HEDGES,8.1656,-75.1063,-72.1037,83.2719,80.2693,2017-03-14,2019-05-06,2019-05-09,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
